# Generacion datos para modelo de MP vs Area y estimación de Biomasa foliar

Se depuran los datos de material particulado(MP) y se agregan con los de area(en este caso area se entiende por la
area que se le midió a las hojas a las que se le midio el MP). Tambien se depuran datos de diametro a altura del 
pecho(DAP) con los que directamente se hace la estimacion de Biomasa foliar en base al modelo de alvarez:

> $B_f=\ln(\beta_1+\beta_0DAP) $

El notebook esta pensado para correrse derecho sin problemas y generar los archivos finales en la carpeta "data" del repositorio en "Datos_para_modelos". El notebook es explicado brevemente. 

Primero se importan librerías de interes, las ultimas 2 librerías son librerias personales con las que se hace 
todo el trabajo de preparacion de datos. Las funciones de esas librerías se utilizan dentro de la funcion "trabajo" q
ue se define en el notebook y es la que hace todo 

In [3]:
import pandas as pd
import numpy as np 
import math
import filtrado_errores as fe
import mapeo_operaciones as mo

# Codigo


## Funciones

### Principal
Funcion que abarca toodas las otras funciones y hace todo el trabajo, recibe la base de datos de MP y DAP, un diccionario que asigna el tipo de hoja por especie y los datos de area por individuo por especie que se calculo
para cada hoja a la que se le midio el material particulado. 

Dentro de la funcion hay parametros de interes, **es de notar el parametro "is_metros"** por defecto se asume que
la funcion de alvarez estaba pensada para recibir datos de DAP en centimetros, asi parece dar resultados mas logicos
ademas de que parece tener mas sentido. **Se mantiene oculta para que no ocupar espacio**

In [4]:
def trabajo(data,tipohojaDict, data_areas):
    data1=data.copy()
    
    H25=-0.0561612903225807 #Factor de correcion de papeles filtro "Horno" 2.5
    H10=-0.0479354838709678 #Factor de correcion de papeles filtro "Horno" 2.5
    NH25=-0.00546666666666667 #Factor de correcion de papeles filtro "Horno" 2.5
    NH10=-0.00366666666666668 #Factor de correcion de papeles filtro "Horno" 2.5
    
    #Funcion Alvarez
    b0=1.424
    b1=5.007
    is_metros=False #Si la funcion de alvarez recibe metros(True) o centimetros(False)
    

    data, reporte1 = fe.filtroInicial(data)
    data = fe.correccion_Horno(data, H25, H10, NH25, NH10)
    
    data = mo.suma_individuo(data)
    
    data = fe.clasficacion_Errores(data)
    data, reporte2 = fe.eliminacion_Errores(data)
    
    data, databiomas = mo.biomasa_individuo(data1,data,b0,b1,metros=is_metros)
    data, reporte3 = mo.altura_Individuo(data1, data)
    
    data = mo.mapeo_tipo_hoja(data,tipohojaDict)

    data.drop(columns=["Errores"],inplace=True)
    data,data_areas= mo.union_mp_dap_areas(data, data_areas)
    
    qc_frame=fe.data_checker(data)
    
    dataEsp=mo.datos_especie(data)

    return data, databiomas, dataEsp

# Manejo de datos

Primero se define el diccionario del tipo de hoja por especie, se mantiene oculto por el momento para que no ocupe mucho espacio en el notebook. 

In [5]:
tipohojaDict={
            'Abarco':'Simple',
            'Acasia':'Compuesta_bipinada',
            'Aguacate':'Simple',
            'Aguacatillo':'Simple',
            'Almendro':'Simple_grande',
            'Arbol del pan':'Hiper_grande_formas_raras',
            'Arizá':'Paripinnada_tallo_hojas',
            'Azuceno':'Simple',
            'Bucaro':'3_foliolos',
            'Caimo':'Simple',
            'Cambulo':'3_foliolos',
            'Caracoli':'Simple_grande',
            'Carambolo':'Paripinnada_tallo_hojas',
            'Carbonero':'Compuesta_bipinada',
            'Casco de vaca':'Simple',
            'Cerezo':'Simple_pequeña',
            'Chiminango':'4_foliolos',
            'Churibio':'4_foliolos',
            'Ciruelo':'Paripinnada_tallo_hojas',
            'Ebano':'Compuesta_bipinada',
            'Eugenio ':'Simple',
            'Falso laurel':'Simple_pequeña',
            'Falso pimiento':'Compuesta_imparipinnada',
            'Frangipan':'Simple',
            'Grosello':'Simple_pequeña',
            'Gualanday':'Compuesta_bipinada',
            'Guayabo':'Simple',
            'Guayacan Amarillo':'Digitada',
            'Guayacan manizaleño':'Simple',
            'Guayacan rosado':'Digitada',
            'Indio desnudo':'Paripinnada_tallo_hojas',
            'Jabutijaba':'Simple_pequeña',
            'Leucaena':'Compuesta_bipinada',
            'Lirata':'Simple_grande',
            'Majagua':'Simple_grande',
            'Mamoncillo':'3_foliolos',
            'Mango':'Simple',
            'Manzano de monte':'3_foliolos',
            'Mulí':'Simple_pequeña',
            'Naranjo':'Simple',
            'Nispero':'Simple',
            'Paco':'Hiper_grande_formas_raras',
            'Pero de agua':'Simple',
            'Saman':'Compuesta_bipinada',
            'Siempre verde':'Compuesta_bipinada',
            'Terminalia':'Simple',
            'Totumo':'Simple',
            'Tulipan':'Paripinnada_tallo_hojas',
            'Urapan':'Compuesta_imparipinnada',
            'Varasanta':'Simple',
            'Velero':'Paripinnada_tallo_hojas',
            'Zapote':'Simple_grande'}

Se importan los datos originales de la carpeta "data/PM_DAP" y se pasan la funcion "trabajo", en el proceso suelta
informacion acerca de los datos.

In [7]:
path_mp="../data/PM_DAP/ResumenUnificada.xlsx"
path_areas="../data/PM_DAP/uniontotalareatot.xlsx"
data=pd.read_excel(path_mp)
data_areas=pd.read_excel(path_areas)
data, databiomas, dataEsp = trabajo(data,tipohojaDict, data_areas)

path_out="../data/Datos_para_modelo/Datos_MP_BiomasaEst_Altura.xlsx"
data.to_excel(path_out,index=False)

--------------------------------------------
Se eliminan 5 datos con problemas: tienen cadenas en los pesos de material particulado
Se preservan 541 datos
--------------------------------------------
Numero de individuos 271
--------------------------------------------
Se detecta la siguiente cantidad de errores:
Material PM 10 negativo = 0
Material PM 2.5 negativo = 0
Falta de datos = 15
--------------------------------------------
Se eliminan 15 individuos con problemas: los clasificados anteriormente
Se preservan 256 individuos
--------------------------------------------
Datos con diametro 389 datos
--------------------------------------------
Individuos con diametro: 251
Individuos sin diametro: 5
--------------------------------------------
Datos con altura calculable: 249
--------------------------------------------
Individuos con altura 236
Individuos sin altura 20
--------------------------------------------
Individuos con tipo de hoja 256
Individuos sin tipo de hoja 0
-------

,Nombre_comun,n_Individuos,Datos_diametro,Datos_altura,Datos_area
1,Acasia,10.0,10.0,10.0,9.0
2,Aguacate,2.0,2.0,2.0,1.0
5,Arbol del pan,2.0,2.0,2.0,1.0
6,Arizá,10.0,10.0,10.0,8.0
10,Cambulo,7.0,6.0,6.0,7.0
14,Casco de vaca,10.0,10.0,10.0,9.0
19,Ebano,3.0,3.0,3.0,2.0
20,Eugenio,9.0,9.0,9.0,0.0
24,Grosello,1.0,0.0,0.0,1.0
27,Guayacan Amarillo,10.0,10.0,10.0,9.0


Se muestra el dataframe para hacer un checkeo de los resultados:

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(data)

,Número del Individuo,Nombre común,Tipo_Hoja,Material_2.5,Material_10,Diametro,BiomasaEstimada,Altura,Area,n_Individuos
0,1,Abarco,Simple,0.012467,0.013667,39.0,5.281664,8.128550,1158.549798,NaN
1,1,Acasia,Compuesta_bipinada,0.022161,0.021335,124.0,6.433409,9.218265,3345.852448,NaN
2,2,Acasia,Compuesta_bipinada,0.013467,0.072935,86-47-77,17.493832,6.722503,1723.846803,NaN
3,3,Acasia,Compuesta_bipinada,0.021361,0.030935,52-40-62-80,22.613297,6.285531,2371.132247,NaN
4,4,Acasia,Compuesta_bipinada,0.039161,0.013667,128,6.465087,5.159819,1118.631024,NaN
5,5,Acasia,Compuesta_bipinada,0.008467,0.020667,65-90-43,17.282021,5.680132,2560.106779,NaN
6,6,Acasia,Compuesta_bipinada,0.034461,0.014667,140-48,12.042454,8.419512,1373.569152,NaN
7,7,Acasia,Compuesta_bipinada,0.015467,0.022135,"87-57-74,5-103,3",23.915544,8.965543,1240.994364,NaN
8,8,Acasia,Compuesta_bipinada,0.036161,0.047667,61-54-77-74-70-53,34.658371,8.689433,2175.132424,NaN
9,9,Acasia,Compuesta_bipinada,0.017467,0.012667,142,6.568665,7.896802,2108.502091,NaN
